### 연습문제
- 위쪽 코드에서 사용했던 네이버웹툰 챌린지 사이트에 대하여 크롤링 진행
- 웹툰명, 작가이름, 별점을 스크래핑
- 총 20페이지를 크롤링하고 스크래핑 진행
- navercomic.csv 로 저장, 본인이름_동적수집.notebook file
- 페이지 변환은 클릭 함수를 사용해서 진행

In [31]:
# 패키지 임포트
import selenium
from selenium import webdriver
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By # 셀레니움 4.0부터 포함된 함수(필수)
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from tqdm import tqdm_notebook
import time

In [32]:
# 크롬 웹드라이버 생성함수
def create_driver():
    service = Service()
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service = service, options = options)
    return driver

In [33]:
driver = create_driver()
url = 'https://comic.naver.com/genre/bestChallenge.nhn'
driver.get(url)

- '//*[@id="content"]/div[3]/button[1]'
- '//*[@id="content"]/div[3]/button[2]'
- '//*[@id="content"]/div[3]/button[3]'
- '//*[@id="content"]/div[3]/button[12]'
  - 10페이지를 한번에 뛰어넘음
-> 여기서 규칙을 찾는다.

- 버튼 번호 1번은 클릭하지 않음
- 버튼 번호 0번은 없음 : 버튼 번호 1 ~ 12
- 버튼 번호 2번은 첫번째 for 문에서는 클릭을 함

- 하지만 두번째 for문 부터는 클릭을 하지 않는다. 이유는 버튼 번호 12번이 버튼 번호 2번을 대신함
- 

In [35]:
result = []

k = 1 # 반복 핸들링 하는 변수
j = -1 # 버튼번호를 정하는 변수

for i in range(1, 40):
    if j == 12:
        k += 3
    j = k % 13
    k = k+1
    if j == 1 or j ==0:
        continue
        
    xpath = '//*[@id="content"]/div[3]/button['+ str(j) + ']'
    
    btn_elem=driver.find_element(By.XPATH,xpath)
    btn_elem.click()
    time.sleep(2)
    sel = '#content > div:nth-child(2) > ul > li > div'
    elem = driver.find_elements(By.CSS_SELECTOR,sel)
    for e in elem :
        res = e.text.split('\n')
        try:
            if len(res) == 7 :
                result.append(pd.DataFrame({'제목':res[1],'작가':res[2],'별점':res[4],'조회수':res[6]},index=range(0,1)))
            else :
                result.append(pd.DataFrame({'제목':res[0],'작가':res[1],'별점':res[3],
                                            '조회수':res[5]},index=range(0,1)))
        except:
            continue
            
    # print(k,'',j,'', xpath)

In [36]:
len(result)

1140

In [38]:
pd.concat(result).reset_index(drop = True).to_csv('crawl_data/webtoon.csv')

In [40]:
pd.read_csv('crawl_data/webtoon.csv', index_col = 0)

,제목,작가,별점,조회수
0,요정이다,stra****,8.83,1.4만
1,미미와 유미의 지구탐험기,라온제나,7.60,"9,648"
2,마피아게임에 어서오세요,함소빌,9.52,55만
3,언제나 반짝이는,주주,9.10,18만
4,런어웨이,하제,9.81,572만
...,...,...,...,...
1135,날 위해 연주해줘,백선,9.16,"6,944"
1136,방구석 해피,itsun,9.63,1.4만
1137,엉뚱한 연애!,나고,8.92,"7,640"
1138,순정 죽이기,이해나,8.79,"5,209"
